# Proyecto

# Libraries

In [24]:
import serial
import sys
import glob
import time
import numpy as np

# Functions

In [20]:
def listSerialPorts():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

def connectToPort(n):
    ports = listSerialPorts()
    return serial.Serial(ports[n])

def readValueFromData(data):
    # +000.000/0 G S
    dataString = data.decode("utf-8").replace('\r\n','')
    if dataString.endswith(' G S'):
        result = dataString.replace(' G S','').replace('/','').replace('+','')
        return float(result)
    else:
        raise Exception("Invalied format")
    
def serialRead(connection):
    while True:
          try:
              connection.flushInput()
              return readValueFromData(connection.readline())
          except:
              pass
    

# Connection

In [21]:

ports = listSerialPorts()

for i in range(0,len(ports)):
    print('{0}: {1}'.format(i, ports[i]))

0: /dev/tty.wlan-debug
1: /dev/tty.BT-163
2: /dev/tty.Bluetooth-Incoming-Port


In [ ]:
port = 3

connection = connectToPort(port)

In [17]:
for i in range(0,10):
    serialRead(connection)
    time.sleep(1)

2.0
0.0001
200.0001
200.0001
200.0
200.0
200.0
200.0
200.0
200.0


In [18]:
connection.close()

# Procedure

Data

In [26]:
scaleDivision = 0.0001 # g
minimumCharge = 0.01 # g
nominalCalibrationMass = 200 # g
EMPs = { # EMP para masa E1 según la OIML R 111
    10: 0.00002,
    20: 0.000025,
    40: 0.00005,
    60: 0.00005,
    80: 0.000075,
    100: 0.00005,
    120: 0.000075,
    140: 0.0001,
    160: 0.0001,
    180: 0.00125,
    200: 0.0001,
    220: 0.000125
}


In [48]:
print('Iniciando procedimiento de calibración de la balanza.')
input('Presione cualquier tecla para continuar.')

points = ['Central','Inferior izquierdo','Superior izquierdo','Superior derecho','Inferior derecho','Central']

print('''
      Paso 1: Prueba de excentricidad
      
      Se debe tomar un patrón de peso de 200 g y utilizarse en todos los puntos de la prueba conforme se indique en pantalla.
      El patrón no se debe cambiar bajo ninguna circunstancia.
      ''')

input('Presione cualquier tecla para continuar.')

eccentricityPoints = []
for i in range(0,len(points)):
    print('\nCalibración del punto {0}, favor posicionar el patrón en el punto {1}'.format(i+1,points[i]))
    keep = True
    currentPoint = []
    while keep:
        value = 190#serialRead(connection)
        if value >= nominalCalibrationMass*0.9:
            keep = False
            currentPoint.append(value)
    print('\nFavor retire el partón de la balanza.')
    keep = True
    while keep:
        value = 0#serialRead(connection)
        if value <= nominalCalibrationMass*0.1:
            keep = False
            currentPoint.append(value)
    
    eccentricityPoints.append(currentPoint)
    
eccentricityPoints = np.array(eccentricityPoints)

p0 = eccentricityPoints[-1:0]

print(p0)

print(eccentricityPoints)

# print((np.vectorize(lambda row: np.abs(row[0]-row[1]-p0)))(eccentricityPoints))

print(np.apply_along_axis(lambda row: np.abs(row[0]-row[1]-p0), 1, eccentricityPoints))

# dEccentricityMax = np.max(np.apply_along_axis(lambda row: np.abs(row[0]-row[1]-p0), 0, eccentricityPoints))

# print(dEccentricityMax)
            
        

Iniciando procedimiento de calibración de la balanza.

      Paso 1: Prueba de excentricidad
      
      Se debe tomar un patrón de peso de 200 g y utilizarse en todos los puntos de la prueba conforme se indique en pantalla.
      El patrón no se debe cambiar bajo ninguna circunstancia.
      

Calibración del punto 1, favor posicionar el patrón en el punto Central

Favor retire el partón de la balanza.

Calibración del punto 2, favor posicionar el patrón en el punto Inferior izquierdo

Favor retire el partón de la balanza.

Calibración del punto 3, favor posicionar el patrón en el punto Superior izquierdo

Favor retire el partón de la balanza.

Calibración del punto 4, favor posicionar el patrón en el punto Superior derecho

Favor retire el partón de la balanza.

Calibración del punto 5, favor posicionar el patrón en el punto Inferior derecho

Favor retire el partón de la balanza.

Calibración del punto 6, favor posicionar el patrón en el punto Central

Favor retire el partón de la b

TypeError: only size-1 arrays can be converted to Python scalars